In [ ]:
%matplotlib inline
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
from keras.layers import Input, Flatten, Dense, Dropout
from keras.callbacks import Callback, ModelCheckpoint
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input

In [ ]:
IMG_WIDTH, IMG_HEIGHT = 150, 150
TRAIN_DATA_DIR = 'data/train'
VALIDATION_DATA_DIR = 'data/validation'
BOTTLENECK_FEATURES_TRAIN_FILE = 'bottleneck_features_train.npy'
BOTTLENECK_FEATURES_VALIDATION_FILE = 'bottleneck_features_validation.npy'
MODEL_WEIGHTS_FILE = 'vgg16-xfer-weights.h5'
NB_TRAIN_SAMPLES = 2000
NB_VALIDATION_SAMPLES = 800
NB_EPOCH = 50

In [ ]:
model_vgg16_conv = VGG16(weights='imagenet', include_top=False)

In [ ]:
datagen = ImageDataGenerator(rescale=1./255)

In [ ]:
generator = datagen.flow_from_directory(
        'data/train',
        target_size=(150, 150),
        batch_size=32,
        class_mode=None,
        shuffle=False)  
bottleneck_features_train = model_vgg16_conv.predict_generator(generator, 2000)
np.save(open(BOTTLENECK_FEATURES_TRAIN_FILE, 'wb'), bottleneck_features_train)

In [ ]:
generator = datagen.flow_from_directory(
        'data/validation',
        target_size=(150, 150),
        batch_size=32,
        class_mode=None,
        shuffle=False)
bottleneck_features_validation = model_vgg16_conv.predict_generator(generator, 800)
np.save(open(BOTTLENECK_FEATURES_VALIDATION_FILE, 'wb'), bottleneck_features_validation)

In [ ]:
train_data = np.load(open(BOTTLENECK_FEATURES_TRAIN_FILE, 'rb'))
train_labels = np.array([1] * (NB_TRAIN_SAMPLES // 2) + [0] * (NB_TRAIN_SAMPLES // 2))
validation_data = np.load(open(BOTTLENECK_FEATURES_VALIDATION_FILE, 'rb'))
validation_labels = np.array([1] * (NB_VALIDATION_SAMPLES // 2) + [0] * (NB_VALIDATION_SAMPLES // 2))

In [ ]:
input = Input(shape=train_data.shape[1:])
x = Flatten(input_shape=train_data.shape[1:])(input)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(1, activation='sigmoid')(x)
model = Model(input=input, output=x)

In [ ]:
model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [ ]:
callbacks = [ModelCheckpoint(MODEL_WEIGHTS_FILE, monitor='val_acc', save_best_only=True)]
history = model.fit(
        train_data, 
        train_labels,
        nb_epoch=NB_EPOCH, 
        batch_size=32,
        validation_data=(validation_data, validation_labels))

In [ ]:
acc = pd.DataFrame({'epoch': [ i + 1 for i in history.epoch ],
                    'training': history.history['acc'],
                    'validation': history.history['val_acc']})
ax = acc.ix[:,:].plot(x='epoch', figsize={5,8}, grid=True)
ax.set_ylabel("accuracy")
ax.set_ylim([0.0,1.0]);